# Using DatabaseConnector

Example followed: https://ohdsi.github.io/DatabaseConnector/articles/Connecting.html

by Anja van Gestel

In [ ]:
## in R:
# remotes::install_github("OHDSI/DatabaseConnector")

%load_ext autoreload
%autoreload 2

import os
os.environ["R_HOME"] = r"C:\Program Files\R\R-4.3.1" # change as needed

from ohdsi import database_connector

# Introduction

This vignette describes how you can use the <code>DatabaseConnector</code> package to connect to a database.

<code>DatabaseConnector</code> supports these database platforms:

* Microsoft SQL Server
* Oracle
* PostgresSql
* Microsoft Parallel Data Warehouse (PDW, a.k.a. Analytics Platform System)
* Amazon Redshift
* Apache Impala
* Google BigQuery
* IBM Netezza
* SQLite
* Spark

For working with SQLite, BigQuery, Netezza and Impala see the original tutorial: 
https://ohdsi.github.io/DatabaseConnector/articles/Connecting.html

# Obtaining drivers

Before <code>DatabaseConnector</code> can be used to connect to a database, the drivers for your platform need to be downloaded to a location in the local file system, which we’ll refer to as the JAR folder.

## The JAR folder

The JAR folder is just a folder in the local file system where the database drivers are stored. It is highly recommended to use the <code>DATABASECONNECTOR_JAR_FOLDER</code> environmental variable to point to this folder, which you can for example set using:

```R
Sys.setenv("DATABASECONNECTOR_JAR_FOLDER" = "c:/temp/jdbcDrivers")
```

Even better would be to add this entry to your <code>.Renviron</code> file:

``` R
DATABASECONNECTOR_JAR_FOLDER = 'c:/temp/jdbcDrivers'
```

That way, the environmental variable will be automatically set whenever you start R.<br>
A convenient way to edit your .Renviron file is by using <code>usethis</code>:

``` R
install.packages("usethis")
usethis::edit_r_environ()
```

If you don’t use the <code>DATABASECONNECTOR_JAR_FOLDER</code> environmental variable, you will need to provide the <code>pathToDriver</code> argument every time you call the <code>downloadJdbcDrivers</code>, <code>connect</code>, <code>dbConnect</code>, or <code>createConnectionDetails</code> functions.

## Where are the JDBC drivers?

If you want to do things manually you can find the JDBC drivers here:<br>
https://ohdsi.github.io/DatabaseConnectorJars/

## Obtaining drivers for SQL Server, Oracle, PostgreSQL, PDW, Spark, RedShift

For your convenience these JDBC drivers are hosted on the OHDSI GitHub pages, and can be downloaded using the <code>downloadJdbcDrivers</code> function.<br>
You’ll first need to specify the JAR folder as described in the previous section, and next download the driver. (for example, for PostgreSQL).

In R this looks like this:

``` R
Sys.setenv("DATABASECONNECTOR_JAR_FOLDER" = "c:/temp/jdbcDrivers")

downloadJdbcDrivers("postgresql")

## DatabaseConnector JDBC drivers downloaded to 'c:/temp/jdbcDrivers'.
```

Note that if we hadn’t specified the <code>DATABASECONNECTOR_JAR_FOLDER</code> environmental variable, we would have to specify the <code>pathToDriver</code> argument when calling <code>downloadJdbcDrivers</code>.

# Creating a connection

On to Python-stuff now!<br>
To connect to a database a number of details need to be specified, such as the database platform, the location of the server, the user name, password, and path to the driver. We can setup a connection with the database using the function <code>connect()</code>. There are different ways to call this function:

1. <code>connect(dbms, user, password, server, port, extraSettings, oracleDriver, pathToDriver)</code>
2. <code>connect(connectionDetails)</code>
3. <code>connect(dbms, connectionString, user, password, pathToDriver)</code>

### Connection with basic parameters

In [ ]:
conn = database_connector.connect(
    dbms = "postgresql",
    server = "localhost/cdm_database",
    user = "postgres",
    password = "password",
    port = 5432
)

See http://ohdsi.github.io/DatabaseConnector/reference/connect.html or type <code>?connect</code> for information on which details are required for each platform. Note that we did not need to specify the <code>pathToDriver</code> argument because we previously already set the <code>DATABASECONNECTOR_JAR_FOLDER</code> environmental variable.

Don’t forget to close any connection afterwards:

In [ ]:
database_connector.disconnect(conn)

### Connection string i.o. server name

<b>Instead of</b> providing the <b>server name</b>, it is also possible to provide the <b>JDBC connection string</b> if this is more convenient:

In [ ]:
conn = database_connector.connect(
    dbms = "postgresql",
    connection_string = "jdbc:postgresql://localhost:5432/cdm_database",
    user = "postgres",
    password = "password"
)

In [ ]:
database_connector.disconnect(conn)

### Connection details

Sometimes we may want to first specify the <b>connection details</b>, and defer connecting until later. This may be convenient for example when the connection is established inside a function, and the details need to be passed as an argument. We can use the <code>createConnectionDetails</code> function for this purpose:

In [ ]:
connection_details = database_connector.create_connection_details(
    dbms = "postgresql",
    server = "localhost/cdm_database",
    user = "postgres",
    password = "password"
)

conn = database_connector.connect(connection_details)

In [ ]:
database_connector.disconnect(conn)

# Using Windows Authentication for SQL Server

In some organizations using Microsoft SQL Server and Windows, it is possible to use Windows Authentication to connect to the server, meaning you won’t have to provide a user name and password, since your Windows credentials will be used. This will require downloading the SQL Server authentication DLL file, and placing it somewhere on your system path. If you don’t have rights to add files to a place on your system path, you can place it anywhere, and set the <code>PATH_TO_AUTH_DLL</code> environmental variable, either using the <code>Sys.setenv()</code>, or by adding it to your <code>.Renviron</code> file. See http://ohdsi.github.io/DatabaseConnector/reference/connect.html#windows-authentication-for-sql-server-1 or type <code>?connect</code> for details on where to get the DLL (and what specific version).